# Census- Employment Status Data

In [1]:
import pandas as pd
import requests

In [2]:
#Census Subject Table API for Employment Status data within Unified School Districts in California for 2018
url="https://api.census.gov/data/2017/acs/acs1/subject?get=group(S2301)&for=school%20district%20(unified)&in=state:06"

In [3]:
#Request for HTTP Data from Census API, which is working <Response [200]>
response = requests.get(url)

In [4]:
#Resetting data from API Data for future formatting, lists data in one column
response_json = response.json()

In [5]:
#Places data in a dataframe and drops index column 0 (with headers), which has 119 schools and 564 columns of variables
unifiedschool_df = pd.DataFrame(response_json,columns=response_json[0]).drop(0)

In [6]:
unifiedschool_df

,GEO_ID,S2301_C01_001E,S2301_C01_001M,S2301_C01_002E,S2301_C01_002M,S2301_C01_003E,S2301_C01_003M,S2301_C01_004E,S2301_C01_004M,S2301_C01_005E,...,S2301_C04_032MA,S2301_C04_032EA,S2301_C04_033MA,S2301_C04_033EA,S2301_C04_034EA,S2301_C04_034MA,S2301_C04_035EA,S2301_C04_035MA,state,school district (unified)
1,9700000US0609070,67426,3815,3007,826,4292,1336,6023,1710,4669,...,None,None,None,None,None,None,None,None,06,09070
2,9700000US0609390,89037,6683,6448,1492,8562,2059,11396,2500,9698,...,None,None,None,None,None,None,None,None,06,09390
3,9700000US0609620,114688,6774,8332,1289,12742,1876,13412,2434,10299,...,None,None,None,None,None,None,None,None,06,09620
4,9700000US0609640,111769,3658,8109,1297,7908,1556,6897,1883,6015,...,None,None,None,None,None,None,None,None,06,09640
5,9700000US0609850,221791,7134,18830,2143,21562,2622,21044,2955,16254,...,None,None,None,None,None,None,None,None,06,09850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,9700000US0641190,124276,5073,7012,1371,10261,1816,12481,2126,11558,...,None,None,None,None,None,None,None,None,06,41190
116,9700000US0643080,49504,2255,3123,941,5038,1426,4905,1054,4250,...,None,None,None,None,None,None,None,None,06,43080
117,9700000US0643470,59048,1753,3937,862,4727,737,6479,915,6152,...,None,None,None,None,None,None,None,None,06,43470
118,9700000US0684500,193438,6729,15743,1748,21477,3016,17555,2537,19004,...,None,None,None,None,None,None,None,None,06,84500


In [7]:
#Find Name column
#unifiedschool_df.to_csv("/Users/nataligracia/git/ca-school-enrollment-trend/Unified2017test.csv")

In [8]:
#View School District to view column titles
unifiedschool_df.loc[unifiedschool_df["NAME"]=="Los Angeles Unified School District, California"]

,GEO_ID,S2301_C01_001E,S2301_C01_001M,S2301_C01_002E,S2301_C01_002M,S2301_C01_003E,S2301_C01_003M,S2301_C01_004E,S2301_C01_004M,S2301_C01_005E,...,S2301_C04_032MA,S2301_C04_032EA,S2301_C04_033MA,S2301_C04_033EA,S2301_C04_034EA,S2301_C04_034MA,S2301_C04_035EA,S2301_C04_035MA,state,school district (unified)
28,9700000US0622710,3872722,15129,239855,7010,358132,6925,438487,7582,398735,...,None,None,None,None,None,None,None,None,06,22710


## _Format Data_

In [9]:
#Census Subject Table Variables for Employment Status data
variableurl = "https://api.census.gov/data/2017/acs/acs1/subject/variables.json"

In [10]:
#Request for HTTP Data from Census API and reset data
variables_json = requests.get(variableurl).json()

In [11]:
#View variable for column title
variables_json["variables"]["S2301_C01_001E"]

{'label': 'Total!!Estimate!!Population 16 years and over',
 'concept': 'EMPLOYMENT STATUS',
 'predicateType': 'int',
 'group': 'S2301',
 'limit': 0,
 'attributes': 'S2301_C01_001EA,S2301_C01_001M,S2301_C01_001MA'}

In [12]:
#Find and replace all columns with variable titles
new_labels = []
for col in unifiedschool_df.columns:
    label = variables_json["variables"].get(col)
    if label is not None:
        label = label['label']
    else:
        if col[-2:] == 'EA':
            label = variables_json["variables"].get(col[:-1])
            label = label['label'] + "||Annotation"
        elif col[-1] == 'M':
            label = variables_json["variables"].get(col[:-1]+'E')
            label = label['label'] + "||MarginOfError"
        elif col[-2:] == 'MA':
            label = variables_json["variables"].get(col[:-2]+'E')
            label = label['label'] + "||MarginOfErrorAnnotation"
    new_labels.append(label)

In [13]:
#Find any columns without titles
new_labels  

['Geography',
 'Total!!Estimate!!Population 16 years and over',
 'Total!!Estimate!!Population 16 years and over||MarginOfError',
 'Total!!Estimate!!AGE!!16 to 19 years',
 'Total!!Estimate!!AGE!!16 to 19 years||MarginOfError',
 'Total!!Estimate!!AGE!!20 to 24 years',
 'Total!!Estimate!!AGE!!20 to 24 years||MarginOfError',
 'Total!!Estimate!!AGE!!25 to 29 years',
 'Total!!Estimate!!AGE!!25 to 29 years||MarginOfError',
 'Total!!Estimate!!AGE!!30 to 34 years',
 'Total!!Estimate!!AGE!!30 to 34 years||MarginOfError',
 'Total!!Estimate!!AGE!!35 to 44 years',
 'Total!!Estimate!!AGE!!35 to 44 years||MarginOfError',
 'Total!!Estimate!!AGE!!45 to 54 years',
 'Total!!Estimate!!AGE!!45 to 54 years||MarginOfError',
 'Total!!Estimate!!AGE!!55 to 59 years',
 'Total!!Estimate!!AGE!!55 to 59 years||MarginOfError',
 'Total!!Estimate!!AGE!!60 to 64 years',
 'Total!!Estimate!!AGE!!60 to 64 years||MarginOfError',
 'Total!!Estimate!!AGE!!65 to 74 years',
 'Total!!Estimate!!AGE!!65 to 74 years||MarginOfError'

In [14]:
#Change column titles for columns labeled "None" 
assert len(new_labels) == len(unifiedschool_df.columns)

In [15]:
#Confirm the number of columns without titles
sum([1 for x in new_labels if x is None])

3

In [16]:
#Setup new Labels of columns labeled "None"
new_labels[-283] = 'NAME'
new_labels[-2] = 'STATE'
new_labels[-1] = 'SCHOOL DISTRICT (UNIFIED)'

In [17]:
#Create new labels of columns labeled "None"
unifiedschool_df.columns = new_labels

In [18]:
#Find all columns without "Annotation" in column title
[col for col in unifiedschool_df.columns if "Annotation" not in col]

['Geography',
 'Total!!Estimate!!Population 16 years and over',
 'Total!!Estimate!!Population 16 years and over||MarginOfError',
 'Total!!Estimate!!AGE!!16 to 19 years',
 'Total!!Estimate!!AGE!!16 to 19 years||MarginOfError',
 'Total!!Estimate!!AGE!!20 to 24 years',
 'Total!!Estimate!!AGE!!20 to 24 years||MarginOfError',
 'Total!!Estimate!!AGE!!25 to 29 years',
 'Total!!Estimate!!AGE!!25 to 29 years||MarginOfError',
 'Total!!Estimate!!AGE!!30 to 34 years',
 'Total!!Estimate!!AGE!!30 to 34 years||MarginOfError',
 'Total!!Estimate!!AGE!!35 to 44 years',
 'Total!!Estimate!!AGE!!35 to 44 years||MarginOfError',
 'Total!!Estimate!!AGE!!45 to 54 years',
 'Total!!Estimate!!AGE!!45 to 54 years||MarginOfError',
 'Total!!Estimate!!AGE!!55 to 59 years',
 'Total!!Estimate!!AGE!!55 to 59 years||MarginOfError',
 'Total!!Estimate!!AGE!!60 to 64 years',
 'Total!!Estimate!!AGE!!60 to 64 years||MarginOfError',
 'Total!!Estimate!!AGE!!65 to 74 years',
 'Total!!Estimate!!AGE!!65 to 74 years||MarginOfError'

In [19]:
#Create a new dataframe for data without the columns that have "Annotation" in the title, which is 284 columns
without_annotation = unifiedschool_df[[col for col in unifiedschool_df.columns if "Annotation" not in col]].copy()

In [20]:
#Find all columns without "MarginOfError" in column title
#[col for col in without_annotation_df.columns if "MarginOfError" not in col]

In [21]:
#Create a new dataframe for data without the columns that have "MarginOfError" in the title, which is 144 columns
withoutmarginerror = without_annotation[[col for col in without_annotation.columns if 'MarginOfError' not in col]].copy()

In [22]:
#Find all columns without "Labor Force Participation Rate" in column title
#[col for col in withoutmarginerror.columns if "Labor Force Participation Rate" not in col]

In [23]:
#Create a new dataframe for data without the columns that have "Labor Force Participation Rate" in the title, which is 109 columns
withoutlaborforce = withoutmarginerror[[col for col in withoutmarginerror.columns if 'Labor Force Participation Rate' not in col]].copy()

In [24]:
#Find all columns without "Sex" in column title
#[col for col in withoutlaborforce.columns if "SEX" not in col]

In [25]:
#Create a new dataframe for data without the columns that have "Sex" in the title, which is 91 columns
withoutsex = withoutlaborforce[[col for col in withoutlaborforce.columns if 'SEX' not in col]].copy()

In [26]:
#Find all columns without "Poverty Status" in column title
#[col for col in withoutsex.columns if "POVERTY STATUS" not in col]

In [27]:
#Create a new dataframe for data without the columns that have "Poverty Status" in the title, which is 85 columns
withoutps = withoutsex[[col for col in withoutsex.columns if 'POVERTY STATUS' not in col]].copy()

In [28]:
#Find all columns without "Disability Status" in column title
#[col for col in withoutps.columns if "DISABILITY STATUS" not in col]

In [29]:
#Create a new dataframe for data without the columns that have "Disability Status" in the title, which is 82 columns
withoutds = withoutps[[col for col in withoutps.columns if 'DISABILITY STATUS' not in col]].copy()

In [30]:
#Find all columns without "Educational Attainment" in column title
#[col for col in withoutds.columns if "EDUCATIONAL ATTAINMENT" not in col]

In [31]:
#Create a new dataframe for data without the columns that have "Educational Attainment" in the title, which is 67 columns
withoutea = withoutds[[col for col in withoutds.columns if 'EDUCATIONAL ATTAINMENT' not in col]].copy()

In [32]:
#Find all columns without "Age" in column title
#[col for col in withoutea.columns if "AGE" not in col]

In [33]:
#Create a new dataframe for data without the columns that have "Age" in the title, which is 37 columns
withoutage = withoutea[[col for col in withoutea.columns if 'AGE' not in col]].copy()

In [34]:
#Format to rename School District Name
def cleandistrict(NAME):
    return NAME.replace(", California","")

In [35]:
#Apply formatting condition to School District Name
withoutage['NAME'].apply(cleandistrict)

1      Coachella Valley Unified School District
2          Colton Joint Unified School District
3               Compton Unified School District
4         Conejo Valley Unified School District
5          Corona-Norco Unified School District
                         ...                   
115               Vista Unified School District
116      Woodland Joint Unified School District
117           Yuba City Unified School District
118              Irvine Unified School District
119           Val Verde Unified School District
Name: NAME, Length: 119, dtype: object

In [36]:
#Create new School District name column with formatting titled District
withoutage['District']= withoutage['NAME'].apply(cleandistrict)

In [37]:
#Find length of Geographt column contents, which is 16
withoutage['Geography'].apply(len).unique()

#Pull a geography
geo = "9700000US0622710"

#Find NCESDist ID
geo.split("US")[1]

'0622710'

In [38]:
#Format to seperate Census Geography code (the state and district ID combined), 
def splitGeo(geo):
    return geo.split("US")[1]

In [39]:
#Apply formatting condition to Geography
withoutage['Geography'].apply(splitGeo)

1      0609070
2      0609390
3      0609620
4      0609640
5      0609850
        ...   
115    0641190
116    0643080
117    0643470
118    0684500
119    0691135
Name: Geography, Length: 119, dtype: object

In [40]:
##Create new Geography name column with formatting titled NCESDist
withoutage['NCESDist']= withoutage['Geography'].apply(splitGeo)

In [41]:
#Add Year column
withoutage['year']= "2017"

In [42]:
withoutage.head()

,Geography,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Total!!Estimate!!Hispanic or Latino origin (of any race),...,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,NAME,STATE,SCHOOL DISTRICT (UNIFIED),District,NCESDist,year
1,9700000US0609070,67426,23574,-999999999,-999999999,-999999999,-999999999,40971,-999999999,53873,...,-999999999.0,17.7,7.7,15.7,"Coachella Valley Unified School District, Cali...",06,09070,Coachella Valley Unified School District,0609070,2017
2,9700000US0609390,89037,51158,-999999999,-999999999,-999999999,-999999999,20559,-999999999,54092,...,-999999999.0,6.1,3.6,5.4,"Colton Joint Unified School District, California",06,09390,Colton Joint Unified School District,0609390,2017
3,9700000US0609620,114688,30612,37214,-999999999,-999999999,-999999999,42859,-999999999,72518,...,-999999999.0,6.0,-999999999.0,7.6,"Compton Unified School District, California",06,09620,Compton Unified School District,0609620,2017
4,9700000US0609640,111769,92016,-999999999,-999999999,11712,-999999999,-999999999,-999999999,16235,...,-999999999.0,2.2,3.7,2.3,"Conejo Valley Unified School District, California",06,09640,Conejo Valley Unified School District,0609640,2017
5,9700000US0609850,221791,124876,18264,-999999999,27848,-999999999,40947,-999999999,87127,...,-999999999.0,5.6,3.6,4.6,"Corona-Norco Unified School District, California",06,09850,Corona-Norco Unified School District,0609850,2017


In [43]:
withoutage.to_csv("/Users/nataligracia/git/ca-school-enrollment-trend/Unified2017.csv")

In [44]:
#Rearrange columns in list
#['NCESDist','District'] + list(without_annotation_df.columns[2:])

In [45]:
#Rearrange columns in dataframe
#final = without_annotation_df[['NCESDist','District','Geography','NAME'] + list(without_annotation_df.columns[2:])]
#final